The aim of the notebook is to help you to take confidence with the undirected graph class of NEMtropy. We will guide you through the functionality of the module and they can be use to reconstruct a network given partial information or to generate a null model given a certain network.
For more detail about the theory behind this, we suggest you to read .....

In [1]:
import numpy as np
import networkx as nx
from NEMtropy import UndirectedGraph, matrix_generator

According to the type of undirected graph, the available models can be divided in:
 - models for binary network: "cm" and "cm_exp"
 - models for weighted networks: "crema", "ecm" and "ecm_exp"

We will make some examples about both classes of models, lets start from binary models.

# 1) Binary Models

In [2]:
# Lets load the zachary karate club from networkx
g = nx.karate_club_graph()

# then we convert it to numpy adjacency matrix, edgelist and degree sequence
adj_kar = nx.to_numpy_array(g)
edg_kar = nx.to_pandas_edgelist(g).values
dseq_kar = np.sum(adj_kar, axis=1)

UndirectedGraph instance can be initialised using adjacency matrix, edgelist or degree and strength sequences. As an example we initialise a graph instance using the adjacency matrix.

In [3]:
graph = UndirectedGraph(adj_kar)

When you use an initiliasition instance that isn't the adjacency matrix you must specify what you are passing to UndirectedGraph. As an exaple we can use the edgelist.

In [4]:
graph = UndirectedGraph(edgelist=edg_kar)

## 1.1) Null Models

Now that our graph instance is initialised lets see what we can do with it. The first example that we consider is how to generate 10 (or more) random versions of zachary karate club having (on average) the same degree sequence of the orginal one. These can be used to compare same properties of the original zachary karate club with their expected value under a random null model preserving the degree sequence.

In [5]:
# Solve tool function solves, for the underlying graph instance, the selected "model" using "method" and "initial_guess" as starting point of the optimization.
# When the undirected network you are considering is binary the available model are 

graph.solve_tool(model="cm_exp",
                 method="newton",
                 initial_guess="random")

# For networks that are not too large (>5000 nodes, but depends on the amount of ram of your laptop) we suggest to use "newton" method.
# You can che if the optimization process was successful by printing the error attribute. 
# Pay attention: error attribute is None if you didn't solve any model yet.

print("The largest error on the degree sequence is ", graph.error)

# As you can notice the error is quite small: the optimisation proccess worked properly!

The largest error on the degree sequence is  3.7787204476558145e-09


In [6]:
# When your netwoork is too large, newton can be too costly in terms of memory and the suggested method is "fixed-point".
graph.solve_tool(model="cm_exp",
                 method="fixed-point",
                 initial_guess="random")

print("The largest error on the degree sequence is ", graph.error)

# As you can notice the error is again quite small. For large networks can be better to look at the relative error on the degree sequence instead of the absolute one.


The largest error on the degree sequence is  6.190089152369183e-08


In [7]:
# After that we have solved the model we can use it to generate random versions of zachary karate club.
# The ensemble sampler function generates "n" random versions of zachary karate club using the model parameters computed by solve_tool.

graph.ensemble_sampler(10, cpu_n=2, output_dir="sample/")

# In the folder "sample/" there are 10 random copies of zachary karate club saved as edgelist.

## 1.2) Graph Reconstruction

Suppose now that you have only partial information about the zachary karate club network: only the degree sequence is disclosed. Following the same procedure described above, we can generate generate a network having (on average) the same observed degree sequence.

In [12]:
# We initialise the graph instance
graph = UndirectedGraph(degree_sequence=dseq_kar)

# We solve the binary model
graph.solve_tool(model="cm_exp",
                 method="newton",
                 initial_guess="random")

# We generate an ensemble copy
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

# We read the edgelist
edgelist_ens = np.loadtxt("sample/0.txt")

# And generate a networkx graph from it
G = nx.Graph()
G.add_edges_from(edgelist_ens)

### ADD network draw

# 2) Weigthed Models

## 2.1) Null Models

In [14]:
# First of all we generate the adjacency matrix of a weighted undirected graph using the matrix_generator module of NEMtropy
adj_weigh = matrix_generator.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=True,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Then we intialise the graph instance
graph = UndirectedGraph(adj_weigh)

# We can the see the graph sequence of the graph by tapping
print(graph.strength_sequence)

[18. 26. 21. 17. 16.]


In [15]:
(adj_weigh - adj_weigh.T).sum()

0.0

In [16]:
# Similarly to the binary case, we can solve the weighted model by solve_tool
graph.solve_tool(model="crema",
                 method="newton",
                 initial_guess="random",
                 adjacency="cm_exp",
                 method_adjacency="newton")

# The optimisation problem is divided in two steps: firstly the binary model is solved and its solution are one
# of the inputs for the weighted model solver. You can choose the method used for the binary and weighted part separately.
# You can even pass an adjacency matrix as imput for the binary part and we will go through this in the next subsection.

In [17]:
# Lets check if the optimisation was successful by printing the errors on the degree and strength sequences
print(graph.error_degree, graph.error_strength)

5.277365211497909e-10 3.1949554113452905e-10


In [18]:
# Similarly to the binary case, we can generate 10 ensembles copies using ensemble_sampler
graph.ensemble_sampler(10, cpu_n=2, output_dir="sample/")

[0 0 0 0 1 1 1 2 2 3] [1 2 3 4 2 3 4 3 4 4] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]), array([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
[0.12958657 0.02618755 0.09328553 0.14433368 0.16086854]
[(0, 1, 2.7582905969109186), (0, 2, 7.570280704591799), (0, 3, 2.546418772148606), (0, 4, 7.796724554408366), (1, 2, 16.67538939329002), (1, 3, 1.000149479854949), (1, 4, 5.21404234570331), (2, 3, 9.647935916645704), (2, 4, 1.0458954006470582), (3, 4, 5.521178013036068)]
[0 0 0 0 1 1 1 2 2 3] [1 2 3 4 2 3 4 3 4 4] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]), array([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
[0.12958657 0.02618755 0.09328553 0.14433368 0.16086854]
[(0, 1, 9.185101735993754), (0, 2, 1.938454715747716), (0, 3, 0.4176162618693704), (0, 4, 6.993109020090586), (1, 2, 4.09723797431318), (1, 3, 1.299168612813763), (1, 4, 2.9609381342554295), (2, 3, 2.120294

## 2.2) Reconstruction

Suppose you have only partial information about the network, for example:
 - The binary structure is well-known and you want to assign weights to network edges;
 - Only marginals information are available: degree and strength sequences.

You can use NEMtropy methods to reconstruct a network configuration that is consistent with the available informations.

### 2.2.1) Adjacency matrix 

In [2]:
# For this example we generate the adjacency matrix of a weighted undirected graph using the matrix_generator module of NEMtropy
adj_weigh = matrix_generator.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=True,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Lets store the binary structure and strength sequence
adj_bin = adj_weigh.astype(bool).astype(float)
strength_seq = adj_weigh.sum(axis=1)

# Now we suppose that the weighted structure is not available
del adj_weigh

# We can initialiase our graph instance with the strength sequence
graph_weighted = UndirectedGraph(strength_sequence=strength_seq)

In [3]:
# Then we solve the model using the adjacency binary matrix as an input for the solver
graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency=adj_bin)


In [5]:
# Lets check if the optimisation was successful by printing the error on strength sequence
print(graph_weighted.error_strength)

2.73587374977069e-10


In [ ]:
# In conclsion we generate a network having the binary structure of "adj_bin" and 
# and weights satisfying (on average) the given "strength_seq"
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

### 2.2.2) Only the marginals are available

In [7]:
# As an example we generate the adjacency matrix of a weighted undirected graph using the matrix_generator module of NEMtropy
adj_weigh = matrix_generator.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=True,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Lets store the binary structure and strength sequence
degree_seq = adj_weigh.astype(bool).astype(float).sum(axis=1)
strength_seq = adj_weigh.sum(axis=1)

# Now we suppose that the weighted structure is not available
del adj_weigh

# We can initialiase our graph instance with degree and strength sequence
graph_weighted = UndirectedGraph(degree_sequence=degree_seq, strength_sequence=strength_seq)

In [9]:
# We solve the model, we pass a model as adjacency and the solver will use degree sequence
# to infer the binary structure
graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="cm_exp",
                          method_adjacency="newton")

In [10]:
# Lets check if the optimisation was successful by printing the errors on strength and degree sequences
print(graph_weighted.error_degree, graph_weighted.error_strength)

1.2218546174835865e-07 6.821210263296962e-13


In [ ]:
# Now we generate a network having a binary structure inferred from dseq
# and weights satisfying (on average) the given "strength_seq"
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

### ADD the part were we read the network and plot it